## Data Preparation

In [1]:
# import necessary modules/libraries

import numpy as  np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import pearsonr

from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=0.7)
sns.set_palette("Greens_r")
#set_palette("Set1", 8, .75) # makes plot lines red

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.preprocessing import Imputer # to impute missing data

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

%matplotlib inline

# read the data
df1 = pd.read_csv('house_price_prediction.csv')
df2 = df1.copy()

# change date col to datetime
df2['date'] = pd.to_datetime(df2['date'])

# change data types of 'waterfront' & 'condition' to categorical
df2[['waterfront', 'condition']] = df1[['waterfront', 'condition']].astype('category')

## Feature Engineering

In [2]:
# splittig statezip into state & zipcode features & removing statezip
df2['state'] = df2['statezip'].apply(lambda x: x.split(' ')[0])
df2['zipcode'] = df2['statezip'].apply(lambda x: int(x.split(' ')[1]))
df2 = df2.drop('statezip', axis='columns')

# creating total_sqft feature: sqft_above + sqft_lot
df2['total_sqft'] = df2.sqft_living + df2.sqft_lot

# creating month feature
df2['month'] = pd.DatetimeIndex(df2['date']).month

df3 = df2.copy()

# creating a column of boolean arrays with 1s for when price is $0 & 0s for when price is not $0
df3['price_is_zero'] = (df3['price'] == 0).astype(int)

# creating a column of boolean arrays with 1s for when yr_renov is 0 & 0s for others
df3['renov_date_is_zero'] = (df3['yr_renovated'] == 0).astype(int)

# replacing 0s with the mean bedroom & bathroom values
df3["bedrooms"].replace({0: round(df3["bedrooms"].mean(), 0)}, inplace=True)
df3["bathrooms"].replace({0: round(df3["bathrooms"].mean(), 0)}, inplace=True)

# removing price outliers
df_no_outs = df3[df3.price < 10000000].copy()

# creating a new dataframe with $0 prices removed to compare later
df_no_zeros = df_no_outs[df_no_outs["price"] != 0].copy()

## Statistical Data Analysis

In [3]:
meanprice_bedrooms = df_no_zeros.groupby('bedrooms')['price'].mean()
medianprice_bedrooms = df_no_zeros.groupby('bedrooms')['price'].median()

meansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].mean()
mediansqft_bedroom = df_no_zeros.groupby("bedrooms")["sqft_living"].median()

may_price = df_no_zeros[df_no_zeros.month==5]['price']
june_price = df_no_zeros[df_no_zeros.month==6]['price']
july_price = df_no_zeros[df_no_zeros.month==7]['price']

def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    n = len(data)

    x = np.sort(data)

    y = np.arange(1, n+1) / n

    return x, y

# Compute ECDFs
x_5, y_5 = ecdf(may_price)
x_6, y_6 = ecdf(june_price)
x_7, y_7 = ecdf(july_price)

### Feature Importance

In [4]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

def feature_importance(X, y, model='reg'):
    score_func = {'reg': f_regression}

    # Score each of the features
    bestfeatures = SelectKBest(score_func=score_func[model], k='all')
    fit = bestfeatures.fit(X, y)

    # Organize and return the scores
    featureScores = pd.DataFrame([X.columns, fit.scores_]).T
    featureScores.columns = ['Feature', 'Score']
    return featureScores.sort_values('Score', ascending=False).set_index('Feature') 

### Hypothesis Test I

**Ho**: Prices of houses with many bedrooms and a few bedrooms are equal.  
**Ha**: Prices of houses with many bedrooms and a few bedrooms are different.

t-test:
I'll compare 2 groups: one with `bedrooms` greater than or equal to 4, and another with less than 4.

In [6]:
many_bedrooms = df_no_outs[df_no_outs['bedrooms'] >= 4]['price']
few_bedrooms = df_no_outs[df_no_outs['bedrooms'] < 4]['price']
stats.ttest_ind(many_bedrooms, few_bedrooms)

Ttest_indResult(statistic=20.612406446603853, pvalue=2.3169170411918228e-90)

#### Result
**t-stat**: 20.6  
**p-val**: 2.3e-90 < 0.05  
We reject the null hypothesis that houses with many bedrooms are priced around the same as houses with fewer bedrooms.

### Hypothesis Test II

**Ho**: Houses with waterfronts are priced the same as houses without waterfronts.  
**Ha**: Houses with waterfronts are priced differently than houses without waterfronts.

In [7]:
waterfront = df_no_outs[df_no_outs['waterfront'] == 1]['price']
no_waterfront = df_no_outs[df_no_outs['waterfront'] == 0]['price']
stats.ttest_ind(waterfront, no_waterfront)

Ttest_indResult(statistic=14.441521571804724, pvalue=2.8817562762538972e-46)

#### Result
**t-stat**: 14.4  
**p-val**: 2.9e-46 < 0.05  
We reject the null hypothesis that houses with waterfronts are priced the same as houses without waterfronts.

# Data Modeling

## Data Preprocessing

In [8]:
df_no_outs = df3[df3.price < 10000000]
df_no_zeros = df_no_outs[df_no_outs["price"] != 0]

# dataframe with both zero prices and outliers.
X = df3.drop(['price', 'date', 'street', 'city', 'state', 'country'], axis='columns')
y = df3["price"]
# splitting data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# dataframe without price outliers
X_no_outs = df_no_outs.drop(['price', 'date', 'street', 'city', 'state', 'country'], axis='columns')
y_no_outs = df_no_outs["price"]
X_train_no_outs, X_test_no_outs, y_train_no_outs, y_test_no_outs = train_test_split(X_no_outs, y_no_outs, test_size = 0.3, random_state=42)

# dataframe without $0 price values
X_no_zeros = df_no_zeros.drop(['price', 'date', 'street', 'city', 'state', 'country'], axis='columns')
y_no_zeros = df_no_zeros["price"]
X_train_no_zeros, X_test_no_zeros, y_train_no_zeros, y_test_no_zeros = train_test_split(X_no_zeros, y_no_zeros, test_size = 0.3, random_state=42)

# dataframe with a parabolic transformation on the `bedrooms` feature
X_transformed = df_no_zeros.drop(['price', 'date', 'street', 'city', 'state', 'country'], axis='columns')
X_transformed['bedrooms_squared'] = X['bedrooms']**2  # parabolic transformation of bedrooms
y_transformed = df_no_zeros['price']
X_train_transformed, X_test_transformed, y_train_transformed, y_test_transformed = train_test_split(X_transformed, y_transformed, test_size = 0.3, random_state=42)

In [9]:
# 4 training sets and test sets
# 3 algorithms

set1 = [X_train, X_test, y_train, y_test]
set2 = [X_train_no_outs, X_test_no_outs, y_train_no_outs, y_test_no_outs]
set3 = [X_train_no_zeros, X_test_no_zeros, y_train_no_zeros, y_test_no_zeros]
set4 = [X_train_transformed, X_test_transformed, y_train_transformed, y_test_transformed]

train_test_sets = [set1, set2, set3, set4]

algorithms = [LinearRegression, Ridge, RandomForestRegressor]
names = ['linear regression', 'ridge regression ', 'random forest    ']

In [10]:
def calc_train_error(X_train, y_train, model):
    '''returns in-sample error for already fit model.'''
    y_pred = model.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    rmse = np.sqrt(mse)
    return rmse
    
def calc_test_error(X_test, y_test, model):
    '''returns out-of-sample error for already fit model.'''
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    return rmse
    
def calc_metrics(X_train, y_train, X_test, y_test, model):
    '''fits model and returns the RMSE for in-sample error and out-of-sample error, and their accuracy score'''
    
    model.fit(X_train, y_train)
    
    train_error = calc_train_error(X_train, y_train, model)
    test_error = calc_test_error(X_test, y_test, model)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    return train_error, test_error, train_score, test_score

### Choosing alpha for Ridge Regression

In [11]:
alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1]

for sets in train_test_sets:
    print('')
    for alpha in alphas:
        model = Ridge(alpha = alpha)
        train_error, test_error, train_score, test_score = calc_metrics(sets[0], sets[2],  sets[1], sets[3], model)
        print('alpha: {} | test_score: {} | test_error: {} | test/train error: {}'.format(alpha, test_score, test_error, round(test_error/train_error, 1)))



alpha: 0.0001 | test_score: 0.06777947773383897 | test_error: 820661.0540042502 | test/train error: 3.4
alpha: 0.001 | test_score: 0.0677794248900705 | test_error: 820661.0772642092 | test/train error: 3.4
alpha: 0.01 | test_score: 0.06777897974205749 | test_error: 820661.273202603 | test/train error: 3.4
alpha: 0.1 | test_score: 0.06777928636739461 | test_error: 820661.1382369914 | test/train error: 3.4
alpha: 1 | test_score: 0.06782834917275116 | test_error: 820639.5422421335 | test/train error: 3.4
alpha: 10.0 | test_score: 0.0676668767071551 | test_error: 820710.6155000443 | test/train error: 3.4

alpha: 0.0001 | test_score: 0.5653871733995002 | test_error: 242953.0513279551 | test/train error: 1.0
alpha: 0.001 | test_score: 0.5653873191173242 | test_error: 242953.01059906863 | test/train error: 1.0
alpha: 0.01 | test_score: 0.5653888828896426 | test_error: 242952.57351617725 | test/train error: 1.0
alpha: 0.1 | test_score: 0.5654117033213278 | test_error: 242946.1949915955 | test

For the 1st & 2nd datasets (df3 & df_no_outs), the best alpha seems to be alpha=1.  
For the 3rd and 4th datasets (df_no_zeros & df_transformed), alpha=0.0001 seems best.

### Choosing parameters for Random Forest

In [12]:
max_depths = [None, 10, 100]
n_estimators = [1, 10, 100, 200]

for sets in train_test_sets:
    print('')
    for max_depth in max_depths:
        for n in n_estimators:
            model = RandomForestRegressor(max_depth=max_depth , n_estimators=n)
            train_error, test_error, train_score, test_score = calc_metrics(sets[0], sets[2],  sets[1], sets[3], model)
            train_error, test_error, train_score, test_score = round(train_error,3), round(test_error,3), round(train_score,3), round(test_score,3)
            print('max_depth: {} | n_estimators: {} | test_score: {} | test_error: {} | test/train error: {}'.format(max_depth, n, test_score, test_error, round(test_error/train_error, 1)))



max_depth: None | n_estimators: 1 | test_score: -0.019 | test_error: 858049.733 | test/train error: 4.3
max_depth: None | n_estimators: 10 | test_score: 0.074 | test_error: 817934.311 | test/train error: 7.4
max_depth: None | n_estimators: 100 | test_score: 0.079 | test_error: 815705.387 | test/train error: 9.3
max_depth: None | n_estimators: 200 | test_score: 0.079 | test_error: 815563.206 | test/train error: 9.4
max_depth: 10 | n_estimators: 1 | test_score: -0.014 | test_error: 856040.201 | test/train error: 3.9
max_depth: 10 | n_estimators: 10 | test_score: 0.074 | test_error: 817735.444 | test/train error: 5.5
max_depth: 10 | n_estimators: 100 | test_score: 0.077 | test_error: 816799.482 | test/train error: 6.0
max_depth: 10 | n_estimators: 200 | test_score: 0.077 | test_error: 816705.971 | test/train error: 6.0
max_depth: 100 | n_estimators: 1 | test_score: 0.023 | test_error: 840353.283 | test/train error: 3.8
max_depth: 100 | n_estimators: 10 | test_score: 0.067 | test_error: 8

Looks like the best values for parameters `max_depth` & `n_estimators` are None & 200 respectively.

In [21]:
lr_train_score = []
lr_test_score = []
lr_train_error = []
lr_test_error = []

ridge_train_score = []
ridge_test_score = []
ridge_train_error = []
ridge_test_error = []

rf_train_score = []
rf_test_score = []
rf_train_error = []
rf_test_error = []

for sets in train_test_sets:
    
    # linear regression
    
    model = algorithms[0]()
    train_error, test_error, train_score, test_score = calc_metrics(sets[0], sets[2], sets[1], sets[3], model)
    train_error, test_error, train_score, test_score = round(train_error, 3), round(test_error, 3), round(train_score, 3), round(test_score, 3)
    
    lr_train_error.append(train_error)
    lr_test_error.append(test_error)
    lr_train_score.append(train_score)
    lr_test_score.append(test_score)
    
    # ridge regression
    
    model = algorithms[1](alpha=0.0001)
    train_error, test_error, train_score, test_score = calc_metrics(sets[0], sets[2], sets[1], sets[3], model)
    train_error, test_error, train_score, test_score = round(train_error, 3), round(test_error, 3), round(train_score, 3), round(test_score, 3)
    
    ridge_train_error.append(train_error)
    ridge_test_error.append(test_error)
    ridge_train_score.append(train_score)
    ridge_test_score.append(test_score)
    
    # random forest
    
    model = algorithms[2](max_depth=None, n_estimators=200)
    train_error, test_error, train_score, test_score = calc_metrics(sets[0], sets[2], sets[1], sets[3], model)
    train_error, test_error, train_score, test_score = round(train_error, 3), round(test_error, 3), round(train_score, 3), round(test_score, 3)
    
    rf_train_error.append(train_error)
    rf_test_error.append(test_error)
    rf_train_score.append(train_score)
    rf_test_score.append(test_score)
    
    '''
    print('train error: {} | test error: {}'.format(train_error, test_error))
    print('test/train: {}'.format(round(test_error/train_error, 1)))
    print(names[0], "R^2: {}, RMSE: {}".format(test_score, rmse))
    '''

In [23]:
features1 = 'all except date, price, street, city, state'
features2 = 'all except date, price, street, city, state'
features3 = 'all except date, price, street, city, state'
features4 = 'all except date, price, street, city, state'
features5 = 'all except date, price, street, city, state'
features6 = 'all except date, price, street, city, state'
features7 = 'all except date, price, street, city, state'
features8 = 'all except date, price, street, city, state'
features9 = 'all except date, price, street, city, state'
features10 = 'all except date, price, street, city, state'
features11 = 'all except date, price, street, city, state'
features12 = 'all except date, price, street, city, state'

content = {'features': [features1, features2, features3, features4, features5, features6, 
                        features7, features8, features9, features10, features11, features12], 
           'model': ['linear regression', 'linear regression', 'linear regression', 'linear regression',
                     'ridge regression', 'ridge regression', 'ridge regression', 'ridge regression', 
                     'random forest', 'random forest', 'random forest', 'random forest'], 
           'test_error': [lr_test_error[0], lr_test_error[1], lr_test_error[2], lr_test_error[3], 
                    ridge_test_error[0], ridge_test_error[1], ridge_test_error[2], ridge_test_error[3], 
                    rf_test_error[0], rf_test_error[1], rf_test_error[2], rf_test_error[3]],
           'train_error': [lr_train_error[0], lr_train_error[1], lr_train_error[2], lr_train_error[3], 
                    ridge_train_error[0], ridge_train_error[1], ridge_train_error[2], ridge_train_error[3], 
                    rf_train_error[0], rf_train_error[1], rf_train_error[2], rf_train_error[3]],
           'test_score': [lr_test_score[0], lr_test_score[1], lr_test_score[2], lr_test_score[3], 
                     ridge_test_score[0], ridge_test_score[1], ridge_test_score[2], ridge_test_score[3],  
                     rf_test_score[0], rf_test_score[1], rf_test_score[2], rf_test_score[3]],
           'train_score': [lr_train_score[0], lr_train_score[1], lr_train_score[2], lr_train_score[3],
                             ridge_train_score[0], ridge_train_score[1], ridge_train_score[2], ridge_train_score[3],
                             rf_train_score[0], rf_train_score[1], rf_train_score[2], rf_train_score[3]],
           'description': ['no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation', 
                           'no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation', 
                           'no transformations', 'no transformations', 'zero prices removed', 'parabolic transformation'],
           'price outs removed': ['no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes']}

table = pd.DataFrame(content, columns=['model', 'test_score', 'train_score', 'test_error', 'train_error', 'price outs removed', 'description', 'features'], index=[1,2,3,4,5,6,7,8,9,10,11,12])
table.sort_values('test_score', ascending=False)

# include hyperparameters, test_RMSE, training_RMSE

,model,test_score,train_score,test_error,train_error,price outs removed,description,features
10,random forest,0.649,0.942,218414.099,89771.290,yes,no transformations,"all except date, price, street, city, state"
12,random forest,0.644,0.946,221871.023,85370.755,yes,parabolic transformation,"all except date, price, street, city, state"
11,random forest,0.635,0.949,224702.115,83128.174,yes,zero prices removed,"all except date, price, street, city, state"
3,linear regression,0.595,0.572,236687.279,239639.587,yes,zero prices removed,"all except date, price, street, city, state"
4,linear regression,0.595,0.572,236648.326,239620.430,yes,parabolic transformation,"all except date, price, street, city, state"
7,ridge regression,0.595,0.572,236687.280,239639.587,yes,zero prices removed,"all except date, price, street, city, state"
8,ridge regression,0.595,0.572,236648.328,239620.430,yes,parabolic transformation,"all except date, price, street, city, state"
2,linear regression,0.565,0.577,242953.056,241694.255,yes,no transformations,"all except date, price, street, city, state"
6,ridge regression,0.565,0.577,242953.051,241694.255,yes,no transformations,"all except date, price, street, city, state"
9,random forest,0.077,0.948,816809.782,86464.164,no,no transformations,"all except date, price, street, city, state"


Model description:
* 
* 
* 
* 
* 

We can see that the best regressor is the Random Forest, with 62% accuracy. It makes sense that the data frame with price outliers and zeros removed had the best performing model.

### CI around my prediction accuracy

calculate standard error RMSE
..

In [15]:
# RMSE == standard deviation of errors
rf_test_error

[816744.468, 230769.646, 246093.763, 231884.757]

In [16]:
one_sd = min(rf_test_error)
two_sd = 2*one_sd
three_sd = 3*one_sd

#### 68% Confidence

In [17]:
min_error = 0 - one_sd
max_error = 0 + one_sd

print('I can be 68% confident that the error in the home price predictions will be between {} and {}.'.format(min_error, max_error))

I can be 68% confident that the error in the home price predictions will be between -230769.646 and 230769.646.


#### 95% Confidence

In [18]:
min_error = 0 - two_sd
max_error = 0 + two_sd

print('I can be 95% confident that the error in the home price predictions will be between {} and {}.'.format(min_error, max_error))

I can be 95% confident that the error in the home price predictions will be between -461539.292 and 461539.292.


### Result Analysis

The following analysis is based on my 68% confidence that the error in the home price predictions is between -218832.738 and 218832.738:

If I've predicted a house to be $\$$400,000 and someone were to buy the house at that price, the worst loss would be \\$181,168 (400,000-218,832), and the highest gain would be \\$618,832 (400,000+218,832).

Let's say my boss wants to buy a house at below my estimate in the hopes of flipping it, i.e., selling the house at around double the price at which he acquired it. With a house that I have estimated to be \\$400,000, if he purchases it at \\$300,000 for example, 3 things can happen:

1. My estimation is correct and he will make a profit of \\$100,000 by just acquiring it, and could make another \\$300,000 by flipping it, making that a total profit of \\$400,000. 
2. The house is actually worth \\$181,168 and he ends up buying at a higher price than what the house is worth, losing \\$118,832 on the purchase.
3. The house is actually worth \\$618,832 and he ends up buying \\$318,832 below the house value, resulting in a 106%  profit just from acquiring it. Meaning, after investing \\$300,000, and expecting to make a profit of \\$200,000-\\$300,000 by flipping it, he ends up making a profit of \\$318,832 instead by just acquiring it, and another \\$300,000 by flipping it, making that a total of \\$618,832 of profit.

Now my boss purchased 100 homes at \$1000 less than my estimated price of \\$400,000:

His **average gain** would be \\$100,000.

His **worst loss** -- with actual price: \\$181,168 (i.e., 400,000-218,832), purchase price: \\$399,000  
\\$21,783,200

    difference between purchase price & actual price: 399,000 - 181,168 = 217,832
    for 100 homes: 217,832*100 = $21,783,200

His **highest gain** -- with actual price: \\$618,832 (i.e., 400,000+218,832), purchase price: \\$399,000  
\\$21,983,200

    difference between actual price & purchase price: 618,832 - 399,000 = 219,832
    for 100 homes 100: 219,832*100 = $21,983,200
    